In [11]:
!git clone -q https: // github.com/AlpaslanErdag/cardata
import random
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Conv2D
import glob
import pandas as pd
import cv2
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import load_model
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam  # - Works
from keras.models import Sequential
import keras
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import imutils

fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference repository
    --refere

In [12]:
colums = ['image_id', 'angle', 'speed']
datadir = 'cardata'
data = pd.read_csv(os.path.join(datadir, 'Test.csv'))
data.head()

modelSpeed = load_model("models/speed_model_0.04521.h5")
modelAngle = load_model("models/angle_model_0.01190.h5")

def load_test_img(datadir, df):
  test_path = []
  
  for i in range(len(data)):
    indexed_data = data.iloc[i]
    image_id = indexed_data[0]
    test_path.append(os.path.join(datadir,image_id))
    test_paths = np.asarray(test_path)
 
  return test_paths

test_paths= load_test_img(datadir + '/test_data/test_data', data)

In [13]:
def angle_preprocess(img):
    #img = img[50:,:,:]
    img = img[int(img.shape[0] / 4):, :, :]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    #img = cv2.resize(img, (200, 66))
    img = (img * 255).astype(np.uint8)
    img = cv2.Canny(img, 100, 200)
    img = cv2.merge((img, img, img))
    img = cv2.resize(img, (200, 66))
    #img = img / 255.0
    #img = np.true_divide(image, 255, dtype=np.float32,casting="unsafe")
    return img


def speed_preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image = cv2.resize(image, (200, 66))
    # hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # lower = np.array([0, 164, 0])
    # upper = np.array([179, 255, 255])
    # mask = cv2.inRange(hsv, lower, upper)
    # image = cv2.bitwise_and(image, image, mask = mask)
    #image = image[int(image.shape[0] / 4):, :, :]
    return image

def batch_generator(test_paths,NumberOfTestData):

  while True:
    speed_img = []
    angle_img = []
    
    for i in range(NumberOfTestData):
      im = mpimg.imread(test_paths[i])
      speed_im = speed_preprocess(im)
      speed_img.append(speed_im)

      angle_im = angle_preprocess(im)
      angle_img.append(angle_im)
          
    yield (np.asarray(angle_img), np.asarray(speed_img))

In [14]:
X_test_angle, X_test_speed = next(batch_generator(test_paths,1020))
speedPred = modelSpeed.predict(X_test_speed)
anglePred = modelAngle.predict(X_test_angle)

2022-04-27 12:48:48.507810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-27 12:48:49.510418: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [30]:
speedPred = np.around(speedPred)
sp = pd.DataFrame(speedPred, index=range(1, 1021))
ap = pd.DataFrame(anglePred, index=range(1, 1021))
df = pd.concat([ap, sp], axis=1)
df.columns = ["angle", "speed"]
df.to_csv("predictions/prediction2704_2.csv")